In [ ]:
%load_ext autoreload
%autoreload 1

%aimport bialignment
import bialignment as ba
import timeit

In [ ]:
args = {'type': 'Protein',
        'gap_cost': -50,
        'gap_opening_cost': -200,
        'shift_cost': -210,
        'structure_weight': 800,
        'max_shift': 1,
        'simmatrix': 'BLOSUM62',
        'nameA': 'Ecoli',
        'nameB': 'Xanthomonas',
        'nodescription': False,
        'outmode': 'full'
       }

## DNA Pol 1 Example

In [ ]:
inputfiles = ['Examples/DNAPolymerase1_Escherichia.cfssp',
         'Examples/DNAPolymerase1_Xanthomonas.cfssp'
        ]
input = [ ba.read_molecule_from_file(f, type="Protein") for f in inputfiles ]

# optionally, truncate input
for x in input:
    for i in range(2):
        x[i] = x[i][:] # define how to truncate here

print(len(input[0][0]))
print(len(input[1][0]))
#print(input)


In [ ]:
remake = False
%store -r stored_alilines
try:
    print(stored_alilines.keys())
except:
    stored_alilines = dict()

for ms in range(3):
    if not remake and (f'max_shift {ms}') in stored_alilines:
        continue
        
    args["max_shift"] = ms

    bialigner = ba.BiAligner(input[0][0],input[1][0],
                             input[0][1],input[1][1], 
                             **args)

    score = timeit.timeit(lambda:bialigner.optimize(),number=1)
    print(score)
    als = list(bialigner.decode_trace_full())
    for i,line in enumerate(alilines):
        print(f"{i:2} {line[0]:12} {line[1]}")

    stored_alilines[(f'max_shift {ms}')] = als
%store stored_alilines

In [ ]:
#computation time: max_shift 0: 0.4 min  (26 s)
print(f"{26.2/60:.2f} min")
#computation time: max_shift 1: 9.4 min
print(f"{626.7/60:.2f} min")
#computation time: max_shift 2: 32.2 min
print(f"{2201.0/60:.2f} min")

In [ ]:
alilines = stored_alilines['max_shift 2']

aliblocks = ba.breaklines(alilines, 80)
for block in aliblocks:
    for i,(name,aliline) in enumerate(block):
        print(f"{i:2} {name:18} {aliline}")
    print()

In [ ]:
for s in range(3):
    alilines = stored_alilines[f'max_shift {s}']
    ba.plot_alignment(alilines, 80, outname=f"dnapoly1-ms{s}-sc-210-sw800.svg")

# Example of Figure 1

In [ ]:
nameA = 'A'
nameB = 'B'
strA = "CHHHHHHHHHHHHHCCCCTCEEEEEEECCTCEEEEEEEECCC"
seqA = "RAKLPLKEKKLTATANYHPGIRYIMTGYSAKYIYSSTYARFR"
seqB = "KAKLPLKEKKLTRTANYHPGIRYIMTGYSAKRIYSSTYAYFR"
strB = "HHHHHHHHHHHHCCCCCCTCEEEEEEECCCCCEEEEEEEECC"

ba.plot_alignment([(nameA, seqA), (nameB, seqB), ('',strA), ('',strB)], 80,
    name_offset=3, show_position_numbers=False, outname = "fig1A.svg")

In [ ]:
seqA1 = "RAKLPLKEKKLTATANYH-PGIRYIMTGYSAK-YIYSSTYARFR"
strA1 = "CHHHHHHHHHHHHHCCCC-TCEEEEEEECCTC-EEEEEEEECCC"
strB1 = "-HHHHHHHHHHHHCCCCCCTCEEEEEEECCCCCEEEEEEEECC-"
seqB1 = "-KAKLPLKEKKLTRTANYHPGIRYIMTGYSAKRIYSSTYAYFR-"

ba.plot_alignment([(nameA, seqA1), (nameB, seqB1), ('',strA1), ('',strB1)], 80,
    name_offset=3, show_position_numbers=False, outname = "fig1B.svg")

In [ ]:
args['nameA'] = 'A'
args['nameB'] = 'B'
args['max_shift'] = 1
args['shift_cost'] = -150
args['structure_weight'] = 800
args['gap_opening_cost'] = -150
args['gap_cost'] = -50

print(args)
print()

bialigner = ba.BiAligner(seqA, seqB, strA, strB,
                         **args)

score = bialigner.optimize()
print('SCORE',score)
print()

alilines = list(bialigner.decode_trace_full())
for i,line in enumerate(alilines):
    print(f"{i:2} {line[0]:18} {line[1]}")
    

In [ ]:
ba.plot_alignment(alilines, 80, show_position_numbers=False,
    name_offset=3, outname = "fig1-shift.svg")

## Parsing dssp and stride output

In [ ]:
import re

def read_dssp_file_content(text, *, chain = None):
    from collections import defaultdict
    res = defaultdict(lambda:"")
    text = text.split('\n')
    text = iter(text)
    # strip header
    for line in text:
        if re.search(r'#  RESIDUE AA STRUCTURE',line):
            break

    for line in text:
        if len(line) < 190:
            continue
                        
        if chain is not None and line[152] != chain:
            continue
        
        res['seq'] += line[13]
        res['str'] += line[16]
            
    res['str'] = res['str'].replace(' ','C')
    return res

def read_dssp_file(filename, **kwargs):
    with open(filename) as fh:
        return read_dssp_file_content(fh.read(), **kwargs)

In [ ]:
def read_stride_file_content(text, *, chain = None):
    from collections import defaultdict
    res = defaultdict(lambda:"")
    text = text.split('\n')
    text = iter(text)

    cur_chain = None
    start = None
    end = None
    for line in text:
        m = re.match(r'^CHN\s+\S+\s+(\w)', line)
        if m:
            cur_chain = m[1]
            
        if chain is not None and cur_chain != chain:
            continue
        
        m = re.search(r'^SEQ\s+(\d+)\s+(\w+)\s+(\d+)', line)
        if m:
            start = int(m[1])
            end = int(m[3])
            res['seq'] += line[10:(end-start+11)]
            
        if re.search(r'^STR', line):
            res['str'] += line[10:(end-start+11)]
            
    res['str'] = res['str'].replace(' ','C')
    return res

def read_stride_file(filename, **kwargs):
    with open(filename) as fh:
        return read_stride_file_content(fh.read(), **kwargs)

## Hydrolase examples

from Bock, 2006

In [ ]:
dsspA = read_dssp_file("Examples/115l.dssp")
dsspB = read_dssp_file("Examples/150l.dssp", chain = "D")


args['nameA'] = '115L'
args['nameB'] = '150L'
args['max_shift'] = 0
args['structure_weight'] = 800

print(args)
print()

print(dsspA['seq'])
print(dsspA['str'])
print(dsspB['seq'])
print(dsspB['str'])

bialigner = ba.BiAligner(dsspA['seq'], dsspB['seq'],
                         dsspA['str'], dsspB['str'],
                         **args)

score = bialigner.optimize()
print('SCORE',score)
print()
alilines = list(bialigner.decode_trace_full())
for i,line in enumerate(alilines):
    print(f"{i:2} {line[0]:18} {line[1]}")
    
ba.plot_alignment(alilines, 80, show_position_numbers=False,
    name_offset=5, outname = "bock06-fig1.svg")

In [ ]:
strideA = read_stride_file("Examples/115l.stride")
strideB = read_stride_file("Examples/150l.stride", chain = "D")

print(strideA['seq'])
print(strideA['str'])
print(strideB['seq'])
print(strideB['str'])

In [ ]:
## If shifts are too cheap, they can be used simply to align one additional structure element!

args['nameA'] = '115L'
args['nameB'] = '150L'
args['max_shift'] = 1
args['shift_cost'] = -100
args['structure_weight'] = 800

print(args)
print()

bialigner = ba.BiAligner(strideA['seq'], strideB['seq'],
                         strideA['str'], strideB['str'],
                         **args)

score = bialigner.optimize()
print('SCORE',score)
print()
alilines = list(bialigner.decode_trace_full())
for i,line in enumerate(alilines):
    print(f"{i:2} {line[0]:18} {line[1]}")
    
ba.plot_alignment(alilines, 60, show_position_numbers=True,
    name_offset=5, outname = "bock06-fig1.svg")

In [ ]:
args['max_shift'] = 0

print(args)
print()

bialigner = ba.BiAligner(strideA['seq'], strideB['seq'],
                         strideA['str'], strideB['str'],
                         **args)

score = bialigner.optimize()
print('SCORE',score)
print()
alilines = list(bialigner.decode_trace_full())
    
ba.plot_alignment(alilines, 60, show_position_numbers=True,
    name_offset=5, outname = "bock06-fig1-noshift.svg")

## Cytochrome P450 Examples

Example from the paper:
https://www.jbc.org/article/S0021-9258(17)50294-X/fulltext

"The structure of this ancestral CYP1B1 with the small, planar α-naphthoflavone permits comparison with structures of all three extant human CYP1B1, CYP1A1, and CYP1A2 enzymes generated with this same ligand (PDB: 3PM0 (26), 4I8V (27), and 2HI4 (28), respectively)."

6OYU is the ancestral CYP1B1 structure from the paper


In [ ]:
cytochrome_name = ["6oyu","3pm0"] #,"2hi4","4i8v"]

In [ ]:
cytochrome_input_stride = [
    read_stride_file(f'Examples/{name}.stride', chain='A')
    for name in cytochrome_name]

cytochrome_input_dssp = [
    read_dssp_file(f'Examples/{name}.dssp', chain='A')
    for name in cytochrome_name]

cytochrome_input = cytochrome_input_dssp

for name, inp in zip(cytochrome_name,cytochrome_input):
    print(name, len(inp['seq']))
    print(inp['seq'])
    print(inp['str'])

In [ ]:
args['max_shift'] = 0
args['shift_cost'] = -210
args['structure_weight'] = 800

print(args)
print()

for a in range(1,2):
    for b in range(0,1):
        nameA = cytochrome_name[a]
        nameB = cytochrome_name[b]
        args['nameA'] = nameA
        args['nameB'] = nameB
        inA =  cytochrome_input[a]
        inB =  cytochrome_input[b]
        
        bialigner = ba.BiAligner(
            inA['seq'], inB['seq'],
            inA['str'], inB['str'],
            **args)

        print(nameA, nameB, len(inA['seq']), len(inB['seq']))
        
        score = bialigner.optimize()
        print('SCORE',score)
        print()
        alilines = list(bialigner.decode_trace_full())
        #for i,line in enumerate(alilines):
        #    print(f"{i:2} {line[0]:18} {line[1]}")

        ba.plot_alignment(alilines, 60, show_position_numbers=True,
            name_offset=6, outname = f"cytochrome-{nameA}-{nameB}.svg")

In [ ]:
## Hand-crafted copy from the paper Figure 1

#           0123456789012345678901234567890123456789
seq_3pm0 = "QAAHLSFARLARRYGDVFQIRLGSCPIVVLNGERAIHQALVQQGSAFADRPSFASFRVVSGGRSMAFGHYSEHWKVQRRAAHSMMRNFFTRQPRSRQVLEGHVLSEARELVALLVRGSADGAFLDPRPLTVVAVANVMSAVCFGCRYSHDDPEFRELLSHNEEFGRTVGAGSLVDVMPWLQYFPNPVRTVFREFEQLNRNFSNFILDKFLRHCESLRPGAAPRDMMDAFILSAEKKAAGDGARLDLENVPATITDIFGASQDTLSTALQWLLLLFTRYPDVQTRVQAELDQVVGRDRLPCMGDQPNLPYVLAFLYEAMRFSSFVPVTIPHATTANTSVLGYHIPKDTVVFVNQWSVNHDPLKWPNPENFDPARFLDKDGLINKDLTSRVMIFSVGKRRCIGEELSKMQLFLFISILAHQCDFRANPNEPAKMNFSYGLTIKPKSFKVNVTLRESMELLD"
str_3pm0 = "CCHHHHHHHHHHHHCCEEEEEETTEEEEEECCHHHHHHHHCCTTTTCCCCCCCHHHHHHHHHTCCCCCCCCHHHHHHHHHHHHHHHHCTTCCTTHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHHHTCCCCCTTCHHHHHHTCCHHHHHHHHCTTCCTTTCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCTTCCCCCCHHHHHHHHHHHHCCCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCHHHHHHHHHHHHHHTCTTCCCCHHHHHHCHHHHHHHHHHHHHHCCCCCCCCEECCCCEEETTEEECTTCEEEECCHHHHCCTTTCCCCCCCCHHHHCCTTCCCCHHHHTTCCCCCCCTTCCCCHHHHHHHHHHHHHHHHHHEEEEECTTCCCCCCEEEECCEEECCCCEEEEECCCCCCCC"

#           0123456789012345678901234567890123456789
seq_6oyu = "SRPPGPFPWPLPHLSFARLARRYGDVFQIRLGSCPVVVLNGERAIRQALVQQGAAFAGRPPFPSFQVVSGGRSLAFGRYSERWKVQRRVAHSTVRAFSTGQPRSRRVLEQHVLGEARELVRLLVRGSAGGAFLDPAPLTVVAVANVMSAVCFGCRYSHDDAEFRGLLSHNEKFGRTVGAGSLVDVLPWLQRFPNPVRTAFRDFQQLNRDFYSFVLDKFLRHRSSLRPGAAPRDMMDAFIHTVPRLDLEYVPATVTDIFGASQDTLSTALQWLLILFTRYPEVQARVQEELDRVVGRDRLPCMDDQPHLPYVMAFLYEAMRFSSFVPVTIPHATTADTSIMGYHIPKDTVVFVNQWSVNHDPVKWPNPEDFNPARFLDNKDLASSVMIFSVGKRRCIGEELSKMQLFLFISILAHQCNFRANPDEDSKMDFSYGLTIKPKSFTINVTLRST"
str_6oyu = "CCCCCCEEECCHHHHHHHHHHHHCCEEEEEETTEEEEEECCHHHHHHHHCCTHHHHCCCCCCCHHHHCCCCTCTTTCCCCCCHHHHHHHHHHHHHTTTCCCHHHHHHHHHHHHHHHHHHHHHHHHHHTTTCCCCHHHHHHHHHHHHHHHHHHCCCCCTTCHHHHHHHHHHHHHHHHHHHHHHHHHCHHHHCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCTTCCCCCHHHHHHHCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCHHHHHHHHHHHHHHHTTCCCCCHHHHHHCHHHHHHHHHHHHHHCCCCCCCCEECCCCEEETTEEECTTCEEEECHHHHHTCTTTCCCTTCCCHHHHCCCHHHHHHCCCCCCHHHCCTTHHHHHHHHHHHHHHHHHHEEEECCTTCCCCCCCCCCCCCCCCCCCEEEEECCC"


#shortened

seq_3pm0 = "AHLSFARLARRYGDVFQIRLGSCPIVVLNGERAIHQALVQQGSAFADRPSFASFRVVSGGRSMAFGHYSEHWKVQRRAAHSMMRNFFTRQPRSRQVLEGHVLSEARELVALLVRGSADGAFLDPRPLTVVAVANVMSAVCFGCRY"
str_3pm0 = "HHHHHHHHHHHHCCEEEEEETTEEEEEECCHHHHHHHHCCTTTTCCCCCCCHHHHHHHHHTCCCCCCCCHHHHHHHHHHHHHHHHCTTCCTTHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHHHTCCCC"

seq_6oyu = "PHLSFARLARRYGDVFQIRLGSCPVVVLNGERAIRQALVQQGAAFAGRPPFPSFQVVSGGRSLAFGRYSERWKVQRRVAHSTVRAFSTGQPRSRRVLEQHVLGEARELVRLLVRGSAGGAFLDPAPLTVVAVANVMSAVCFGCRY"
str_6oyu = "HHHHHHHHHHHHCCEEEEEETTEEEEEECCHHHHHHHHCCTCCCCCCCCCCHHHCCHHHHTCTTTCCCCCCHHHHHHHHHHHHHTTTCCCHHHHHCHHHHHHHHHHHHHHHHHHHHTTTCCCCHHHHHHHHHHHHHHHHHHCCCC"

str_3pm0 = str_3pm0.replace("T","C")
str_6oyu = str_6oyu.replace("T","C")



In [ ]:
args['max_shift'] = 2
args['shift_cost'] = -210
args['structure_weight'] = 800


nameA = "Human 1B1"
nameB = "N98 1B1_M"

args['nameA'] = nameA
args['nameB'] = nameB
inA =  {'seq': seq_3pm0, 'str': str_3pm0}
inB =  {'seq': seq_6oyu, 'str': str_6oyu}

print(nameA, nameB, len(inA['seq']), len(inB['seq']))


for ms in range(3):
    # ==============================
    # max shift 0
    args['max_shift'] = ms
    
    bialigner = ba.BiAligner(
        inA['seq'], inB['seq'],
        inA['str'], inB['str'],
        **args)

    score = bialigner.optimize()
    print('SCORE',score)
    print()
    alilines = list(bialigner.decode_trace_full())
    #for i,line in enumerate(alilines):
    #    print(f"{i:2} {line[0]:18} {line[1]}")

    ba.plot_alignment(alilines, 75, show_position_numbers=True,
        name_offset=10, outname = f"cytochrome-{nameA}-{nameB}-maxshift-{ms}.svg")
